# Ejercicio 3

El decano Esteban está organizando las comisiones de la materia Bases de Datos.
Actualmente tiene 3 comisiones, con 3 profesores, Alberto, Bruno y Carla.
La materia se dicta 2 veces a la semana, y solamente hay algunos horarios disponibles.
Antes de que empiece el cuatrimestre, les pidió a cada uno de los profesores que indiquen
qué tan bien (del 1 al 5, siendo 5 lo más conveniente para el profesor) les queda cada
horario para dictar clases. Estas fueron sus respuestas:

|         | Lunes 18hs | Miercoles 14hs | Miercoles 16hs | Jueves 8hs | Jueves 10hs | Jueves 20hs |
|---------|------------|----------------|----------------|------------|-------------|-------------|
| Alberto | 3          | 5              | 4              | 3          | 4           | 3           |
| Bruno   | 1          | 4              | 3              | 4          | 2           | 1           |
| Carla   | 2          | 4              | 3              | 1          | 3           | 3           |


Cómo debería el decano organizar las comisiones para maximizar la suma de las
conveniencias de los profesores? Indique los horarios de cada comisión.

In [36]:
import pandas as pd
from ortools.linear_solver import pywraplp

In [37]:
data = pd.DataFrame(
     [[3,5,4,3,4,3],
     [1,4,3,4,2,1],
     [2,4,3,1,3,3]], 
     columns=['L18', 'M14', 'M16', 'J8', 'J10', 'J20'],
     index=['Alberto', 'Bruno', 'Carla'])

data

,L18,M14,M16,J8,J10,J20
Alberto,3,5,4,3,4,3
Bruno,1,4,3,4,2,1
Carla,2,4,3,1,3,3


In [38]:
solver = pywraplp.Solver.CreateSolver('CBC')

In [39]:
profesores = data.index.values
n_profesores = len(profesores)

horarios = data.columns.values
n_horarios = len(horarios)

## Variables de decisión

In [40]:
variables = {}

for p in profesores:
    for h in horarios:
        v = solver.BoolVar(f'X_{p}_{h}')
        variables[(p, h)] = v

variables

{('Alberto', 'L18'): X_Alberto_L18,
 ('Alberto', 'M14'): X_Alberto_M14,
 ('Alberto', 'M16'): X_Alberto_M16,
 ('Alberto', 'J8'): X_Alberto_J8,
 ('Alberto', 'J10'): X_Alberto_J10,
 ('Alberto', 'J20'): X_Alberto_J20,
 ('Bruno', 'L18'): X_Bruno_L18,
 ('Bruno', 'M14'): X_Bruno_M14,
 ('Bruno', 'M16'): X_Bruno_M16,
 ('Bruno', 'J8'): X_Bruno_J8,
 ('Bruno', 'J10'): X_Bruno_J10,
 ('Bruno', 'J20'): X_Bruno_J20,
 ('Carla', 'L18'): X_Carla_L18,
 ('Carla', 'M14'): X_Carla_M14,
 ('Carla', 'M16'): X_Carla_M16,
 ('Carla', 'J8'): X_Carla_J8,
 ('Carla', 'J10'): X_Carla_J10,
 ('Carla', 'J20'): X_Carla_J20}

## Restricciones

In [41]:
# Maxima cantidad de clases por profesor
for p in profesores:
    ct = solver.Constraint(0, 2, f'Máxima cantidad de clases para para el profesor {p}')
    for h in horarios:
        ct.SetCoefficient(variables[(p, h)], 1)

# Maxima cantidad de profesores por clase
for h in horarios:
    ct = solver.Constraint(0, 1, f'Máxima cantidad de profesores para la clase {h}')
    for p in profesores:
        ct.SetCoefficient(variables[(p, h)], 1)

## Función Objetivo

In [43]:
objective = solver.Objective()
for h in horarios:
    for p in profesores:
        objective.SetCoefficient(variables[(p, h)], int(data.loc[p,h]))
objective.SetMaximization()

In [44]:
print(solver.ExportModelAsLpFormat(False).replace('\\', '').replace(',_', ','), sep='\n')

 Generated by MPModelProtoExporter
   Name             : 
   Format           : Free
   Constraints      : 9
   Variables        : 18
     Binary         : 18
     Integer        : 0
     Continuous     : 0
Maximize
 Obj: +3 X_Alberto_L18 +5 X_Alberto_M14 +4 X_Alberto_M16 +3 X_Alberto_J8 +4 X_Alberto_J10 +3 X_Alberto_J20 +1 X_Bruno_L18 +4 X_Bruno_M14 +3 X_Bruno_M16 +4 X_Bruno_J8 +2 X_Bruno_J10 +1 X_Bruno_J20 +2 X_Carla_L18 +4 X_Carla_M14 +3 X_Carla_M16 +1 X_Carla_J8 +3 X_Carla_J10 +3 X_Carla_J20 
Subject to
 Máxima_cantidad_de_clases_para_para_el_profesor_Alberto_rhs: +1 X_Alberto_L18 +1 X_Alberto_M14 +1 X_Alberto_M16 +1 X_Alberto_J8 +1 X_Alberto_J10 +1 X_Alberto_J20  <= 2
 Máxima_cantidad_de_clases_para_para_el_profesor_Alberto_lhs: +1 X_Alberto_L18 +1 X_Alberto_M14 +1 X_Alberto_M16 +1 X_Alberto_J8 +1 X_Alberto_J10 +1 X_Alberto_J20  >= 0
 Máxima_cantidad_de_clases_para_para_el_profesor_Bruno_rhs: +1 X_Bruno_L18 +1 X_Bruno_M14 +1 X_Bruno_M16 +1 X_Bruno_J8 +1 X_Bruno_J10 +1 X_Bruno_J20 

In [45]:
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('El problema tiene una solución óptima')
else:
    print('No se encontró una solución óptima')

El problema tiene una solución óptima


In [46]:
total = solver.Objective().Value()
print(f'La conveniencia total es {total}')

La conveniencia total es 21.0


In [47]:
# Imprimo las variables con valor a 1 en la solución óptima:
for variable in solver.variables():
    if variable.solution_value() == 1:
        print(variable)

X_Alberto_L18
X_Alberto_J10
X_Bruno_M16
X_Bruno_J8
X_Carla_M14
X_Carla_J20


In [48]:
solution = data.copy()
for h in horarios:
    for p in profesores:
        solution.loc[p, h] = variables[p, h].solution_value()
solution

,L18,M14,M16,J8,J10,J20
Alberto,1,0,0,0,1,0
Bruno,0,0,1,1,0,0
Carla,0,1,0,0,0,1


In [ ]:
def dataframe_to_markdown(df):
    """
    Convert a Pandas DataFrame to a Markdown table.
    
    Parameters:
    df (pd.DataFrame): The Pandas DataFrame to convert.
    
    Returns:
    str: Markdown string representing the DataFrame as a table.
    """
    
    markdown_table = "| " + " | ".join(["Index"] + list(df.columns)) + " |\n"
    markdown_table += "| " + " | ".join(["---"] * (len(df.columns) + 1)) + " |\n"

    for index, row in df.iterrows():
        markdown_table += f"| {index} | " + " | ".join(map(str, row)) + " |\n"

    return markdown_table

dataframe_to_markdown(solution)

| Index | L18 | M14 | M16 | J8 | J10 | J20 |
| --- | --- | --- | --- | --- | --- | --- |
| Alberto | 1 | 0 | 0 | 0 | 1 | 0 |
| Bruno | 0 | 0 | 1 | 1 | 0 | 0 |
| Carla | 0 | 1 | 0 | 0 | 0 | 1 |